In [4]:
import os
import numpy as np
from PIL import Image

In [27]:
#Parameters

image_size = (64,64)
data_directory = "Images"

def loading_images(data_directory, image_size):
    X = []
    y = []
    # Label each folder as either 0 or 1
    labels = os.listdir(data_directory)
    label_dict = {label: i for i, label in enumerate(labels)}
    
    
    for label in labels:
        folder_1 = os.path.join(data_directory, label)
        data_dire = os.listdir(folder_1)

        #folder_2 = ""
        for d in data_dire:
            
            if d=="train":
                folder_2 = os.path.join(folder_1,d)
                Files = os.listdir(folder_2)
                
                
                for filename in Files:
                    if filename.endswith("jpg"):
                        image_path = os.path.join(folder_2, filename)
                        img = Image.open(image_path).convert("RGB")
                        img_resized = img.resize(image_size)
                        X_array = np.array(img_resized)
                        X.append(X_array.flatten())
                        y.append(label_dict[label])
    return X, y
    

In [28]:
loading_images("../Images", (64,64))

([array([255, 255, 255, ..., 246, 239, 221], dtype=uint8),
  array([ 59,  49,  39, ..., 104,  79,  58], dtype=uint8),
  array([ 39,  83, 138, ..., 128, 225,  90], dtype=uint8),
  array([239, 238, 219, ...,  69,  43,  30], dtype=uint8),
  array([190, 198, 200, ..., 216, 224, 226], dtype=uint8),
  array([187, 194, 201, ..., 206, 212, 217], dtype=uint8),
  array([40, 35, 29, ..., 39, 35, 32], dtype=uint8),
  array([159, 169, 181, ..., 168, 132, 105], dtype=uint8),
  array([ 36,  37,  42, ..., 109, 102,  88], dtype=uint8),
  array([255, 255, 255, ..., 255, 255, 255], dtype=uint8),
  array([198, 154, 117, ..., 162, 138, 120], dtype=uint8),
  array([188, 190, 189, ..., 180, 182, 181], dtype=uint8),
  array([191, 193, 205, ..., 173, 180, 189], dtype=uint8),
  array([216, 214, 225, ..., 188, 146, 161], dtype=uint8),
  array([95, 94, 50, ..., 73, 57, 39], dtype=uint8),
  array([20, 20, 20, ..., 63, 62, 53], dtype=uint8),
  array([165, 203, 241, ..., 170, 177, 189], dtype=uint8),
  array([210, 2